<a href="https://colab.research.google.com/github/venky1845/pythonproject/blob/main/Copy_of_Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests

def fetch_papers(query: str):
    """
    Fetch paper IDs from PubMed using a search query.
    """
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
        "db": "pubmed",
        "term": query,
        "retmode": "json",
        "retmax": 100  # Fetch up to 100 papers for simplicity
    }

    response = requests.get(base_url, params=params)
    response.raise_for_status()  # Raise an error for bad responses
    data = response.json()
    return data.get("esearchresult", {}).get("idlist", [])


In [11]:
import xml.etree.ElementTree as ET
from xml.etree import ElementTree
def fetch_paper_details(id):
  filtered_papers = []
  non_academic_authors = []
  company_affiliations = []
  params = {"db":"pubmed","id":id, "retmode":"xml"}
  base_url = "http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
  response = requests.get(base_url, params)
  root = ElementTree.fromstring(response.content)
  for article in root.findall("PubmedArticle"):
    title = article.find('.//ArticleTitle')
    date = article.find('.//ArticleDate')
    year = date.find('.//Year')
    month = date.find('.//Month')
    day = date.find('.//Day')
    print(year.text+ month.text+day.text)
    authors = article.find(".//AuthorList")
    if(authors is None):
      continue
    if(len(authors) == 0):
      continue
    for author in authors:
      lastName = author.find(".//LastName").text
      foreName = author.find(".//ForeName").text
      Intial = author.find(".//Initials").text
      afList = author.find('.//AffiliationInfo')
      name = lastName+"   "+foreName+"   "+Intial
      if(afList is None):
        continue
      if(len(afList) == 0):
        continue
      for affiliation in afList:
        if affiliation is not None and "Inc" in affiliation or "LLC" in affiliation or "Corp" in affiliation:
          company_affiliations.append(affiliation.text)
      if len(company_affiliations) != 0:
        non_academic_authors.append(name)
    if non_academic_authors:
          filtered_papers.append({
                "PubmedID": id,
                "Title": title,
                "Publication Date": year.text+ month.text+day.text,
                "Non-academic Author(s)": ", ".join(non_academic_authors),
                "Company Affiliation(s)": ", ".join(company_affiliations)
            })
    return filtered_papers



In [12]:
"""
ids = [ 39847825 ,  39847811 ,  39847800 ,  39847788 ,  39847771 ,  39847761 ,  39847746 ,  39847745 ,  39847744 ,  39847743 ,  39847736 ,  39847732 ,  39847613 ,  39847598 ,  39847593 ,  39847582 ,  39847581 ,  39847579 ,  39847533 ,  39847517 ,  39847514 ,  39847509 ,  39847504 ,  39847469 ,  39847462 ,  39847453 ,  39847434 ,  39847385 ,  39847372 ,  39847363 ,  39847350 ,  39847346 ,  39847292 ,  39847288 ,  39847284 ,  39847282 ,  39847281 ,  39847267 ,  39847248 ,  39847239 ,  39847238 ,  39847237 ,  39847233 ,  39847232 ,  39847226 ,  39847224 ,  39847205 ,  39847203 ,  39847190 ,  39847188 ,  39847185 ,  39847178 ,  39847175 ,  39847172 ,  39847168 ,  39847163 ,  39847154 ,  39847153 ,  39847145 ,  39847143 ,  39847133 ,  39847131 ,  39847129 ,  39847119 ,  39847116 ,  39847110 ,  39847107 ,  39847099 ,  39847077 ,  39847074 ,  39847055 ,  39846985 ,  39846984 ,  39846983 ,  39846982 ,  39846981 ,  39846970 ,  39846966 ,  39846957 ,  39846954 ,  39846951 ,  39846879 ,  39846810 ,  39846804 ,  39846802 ,  39846761 ,  39846674 ,  39846615 ,  39846613 ,  39846611 ,  39846608 ,  39846549 ,  39846541 ,  39846480 ,  39846465 ,  39846444 ,  39846434 ,  39846431 ,  39846417 ,  39846413 ]
for id in ids:
  fetch_paper_details(id)
"""
id = 39847736
ar = fetch_paper_details(id)


20250124


In [13]:
def filter_non_academic_papers(papers):
    """
    Filter papers with at least one author affiliated with a non-academic organization.
    """
    filtered_papers = []
    for paper in papers:
        non_academic_authors = []
        company_affiliations = []

        for author in paper["Authors"]:
            affiliation = author.get("affiliation", "")
            # Check for company-like keywords in affiliations
            if "Inc" in affiliation or "LLC" in affiliation or "Corp" in affiliation:
                non_academic_authors.append(author.get("name", ""))
                company_affiliations.append(affiliation)

        if non_academic_authors:
            filtered_papers.append({
                "PubmedID": paper["PubmedID"],
                "Title": paper["Title"],
                "Publication Date": paper["Publication Date"],
                "Non-academic Author(s)": ", ".join(non_academic_authors),
                "Company Affiliation(s)": ", ".join(company_affiliations)
            })

    return filtered_papers


In [14]:
import pandas as pd

def save_to_csv(papers, filename="results.csv"):
    """
    Save the filtered paper data to a CSV file.
    """
    df = pd.DataFrame(papers)
    df.to_csv(filename, index=False)
    print(f"Results saved to {filename}.")


In [15]:
def main(query, filename="results.csv"):
    print(f"Fetching papers for query: {query}...")
    pubmed_ids = fetch_papers(query)
    print(f"Fetched {len(pubmed_ids)} paper IDs.")

    if not pubmed_ids:
        print("No papers found.")
        return

    print("Fetching paper details...")
    filtered_papers = []
    for pubMedid in pubmed_ids:
      filtered_papers = fetch_paper_details(pubMedid)
      if not filtered_papers:
          print("No papers found with non-academic authors.")
          return
      if filtered_papers:
        print(f"Found {len(filtered_papers)} papers with non-academic authors.")
        save_to_csv(filtered_papers, filename)


In [16]:
# Example usage
query = "cancer treatment"
filename = "sample.csv"

main(query, filename)


Fetching papers for query: cancer treatment...
Fetched 100 paper IDs.
Fetching paper details...
20250124
No papers found with non-academic authors.
